# Projeto 1 - Ciência dos Dados

Nome: Alberto Mansur

Nome: Lior Lerner

Nome: Raymond Diwan


Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [220]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI

In [221]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\betoa\OneDrive\INSPER\2 sem\CDADOS\PROJETO 1 OF\P1-Cdados-main


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [222]:
filename = 'tweets_xbox.xlsx'

In [223]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Unnamed: 1
0,"this isn’t meant to be a console war tweet, it...",0
1,spot the props! here's some the of models were...,0
2,@jaydakid8 i had both the ps5 and xbox series ...,0
3,the #playstationshowcase was a great success \...,0
4,my brother finally got a new xbox but some thi...,2


In [224]:
train = train.rename(columns={'Unnamed: 1': 'Indíce'})

In [225]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,@swtheory66 i've got ps4 and xbox series s and...,1,NaN,NaN,NaN,NaN,NaN
1,"@zdzisiu2007 i don't be seeing ""to xbox"" anywh...",0,NaN,NaN,NaN,NaN,NaN
2,jorge henrique played ufc 4 (xbox one) in the ...,1,NaN,NaN,NaN,NaN,NaN
3,xbox news: meet the athletes of the gaming for...,0,NaN,NaN,NaN,NaN,NaN
4,controversial but if you own siege on pc and x...,0,NaN,NaN,NaN,NaN,NaN


In [226]:
test = test.rename(columns={'Unnamed: 1': 'Indíce'})

___
## Classificador automático de sentimento


* Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


O produto escolhido foi o Xbox, que é um console de videogame da Microsoft.


Muito relevante (3):

Os tweets considerados Muito Relevantes foram os que envolviam problemas técnicos com o console e falta de estoque

Relevantes (2):
Os tweets considerados relevantes foram os que envolviam opiniões  das pessoas em relação à comparação entre o Xbox e seus principais competidores, e opiniões relevantes de como melhorar a plataforma

Pouco Relevantes (1):
Os tweets considerados pouco relevantes foram os que envolviam ideias de jogos que poderião ser adicionados e opiniões relativamente relevantes (se a pessoas gosta da plataforma, etc.)

Irrelevantes (0):
Os tweets considerados irrelevantes foram os que não atenderam nenhuma das categorias listadas acima




___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [227]:
#Definindo as variáveis como categóricas
train['Indíce'] = train['Indíce'].astype('category')
test['Indíce'] = test['Indíce'].astype('category')
train['Indíce'].cat.categories = ['Não relevante','Pouco relevante','Relevante','Muito relevante']  
test['Indíce'].cat.categories = ['Não relevante','Pouco relevante','Relevante','Muito relevante']  


In [228]:
test.head()

,Teste,Indíce,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,@swtheory66 i've got ps4 and xbox series s and...,Pouco relevante,NaN,NaN,NaN,NaN,NaN
1,"@zdzisiu2007 i don't be seeing ""to xbox"" anywh...",Não relevante,NaN,NaN,NaN,NaN,NaN
2,jorge henrique played ufc 4 (xbox one) in the ...,Pouco relevante,NaN,NaN,NaN,NaN,NaN
3,xbox news: meet the athletes of the gaming for...,Não relevante,NaN,NaN,NaN,NaN,NaN
4,controversial but if you own siege on pc and x...,Não relevante,NaN,NaN,NaN,NaN,NaN


In [229]:
def emoji(text):
    pattern = ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip()
    return pattern

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;/@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', emoji(text))
    return text_subbed    



In [230]:
def treinamento(data):
    p_relevantes = dict()
    p_irrelevantes = dict()
    p_possiveis = set()
    for index,row in data.iterrows():
        tweet = row.Treinamento
        for palavra in cleanup(tweet).split():
            p_possiveis.add(palavra)
            if row['Indíce'] == 'Muito relevante':
                p_relevantes[palavra] = p_relevantes.get(palavra,0)+1
            else:
                p_irrelevantes[palavra] = p_irrelevantes.get(palavra,0)+1
    return (p_possiveis,p_relevantes,p_irrelevantes)

In [231]:
resultados_treino = train['Indíce'].value_counts(True)
muito_relevantes_treino = resultados_treino[3]
relevantes_treino = resultados_treino[2]
pouco_relevantes_treino = resultados_treino[1]
não_relevantes_treino = resultados_treino[0]

resultados_brutos_treino = train['Indíce'].value_counts()
muito_relevantes_brutos_treino = resultados_brutos_treino[3]
relevantes_brutos_treino = resultados_brutos_treino[2]
pouco_relevantes_brutos_treino = resultados_brutos_treino[1]
não_relevantes_brutos_treino = resultados_brutos_treino[0]

tweets_totais_treino = muito_relevantes_brutos_treino + relevantes_brutos_treino + pouco_relevantes_brutos_treino + não_relevantes_brutos_treino
resultados_teste = test['Indíce'].value_counts()
muito_relevantes_teste = resultados_teste[3]
relevantes_teste = resultados_teste[2]
pouco_relevantes_teste = resultados_teste[1]
não_relevantes_teste = resultados_teste[0]

resultados_brutos_teste = test['Indíce'].value_counts()
muito_relevantes_brutos_teste = resultados_brutos_teste[3]
relevantes_brutos_teste = resultados_brutos_teste[2]
pouco_relevantes_brutos_teste = resultados_brutos_teste[1]
não_relevantes_brutos_teste = resultados_brutos_teste[0]

tweets_totais_teste = muito_relevantes_brutos_teste + relevantes_brutos_teste + pouco_relevantes_brutos_teste + não_relevantes_brutos_teste

print('Tweets Muito relevante em Treinamento: {0}'.format(muito_relevantes_brutos_treino))
print('Tweets relevantes em Treinamento: {0}'.format(relevantes_brutos_treino))
print('Tweets Pouco relevante em Treinamento: {0}'.format(pouco_relevantes_brutos_treino))
print('Tweets Não relevante em Treinamento: {0}'.format(não_relevantes_brutos_treino))
print('Tweets totais em Treinamento: {0}'.format(tweets_totais_treino))
print('\n')


print('Porcentagem de Tweets Muito relevante em Treinamento: {0:.2f}%'.format(muito_relevantes_treino*100))
print('Porcentagem de Tweets relevantes em Treinamento: {0:.2f}%'.format(relevantes_treino*100))
print('Porcentagem de Tweets Pouco relevante em Treinamento: {0:.2f}%'.format(pouco_relevantes_treino*100))
print('Porcentagem de Tweets Não relevante em Treinamento: {0:.2f}%'.format(não_relevantes_treino*100))
print('\n')

print('Tweets Muito relevante em Teste: {0}'.format(muito_relevantes_brutos_teste))
print('Tweets relevantes em Teste: {0}'.format(relevantes_brutos_teste))
print('Tweets Pouco relevante em Teste: {0}'.format(pouco_relevantes_brutos_teste))
print('Tweets Não relevante em Teste: {0}'.format(não_relevantes_brutos_teste))
print('Tweets totais em Teste: {0}'.format(tweets_totais_teste))
print('\n')

print('Porcentagem de Tweets Muito relevante em Teste: {0:.2f}%'.format(muito_relevantes_brutos_teste*100/250))
print('Porcentagem de Tweets relevantes em Teste: {0:.2f}%'.format(relevantes_brutos_teste*100/250))
print('Porcentagem de Tweets Pouco relevante em Teste: {0:.2f}%'.format(pouco_relevantes_brutos_teste*100/250))
print('Porcentagem de Tweets Não relevante em Teste: {0:.2f}%'.format(não_relevantes_brutos_teste*100/250))

Tweets Muito relevante em Treinamento: 11
Tweets relevantes em Treinamento: 61
Tweets Pouco relevante em Treinamento: 79
Tweets Não relevante em Treinamento: 349
Tweets totais em Treinamento: 500


Porcentagem de Tweets Muito relevante em Treinamento: 2.20%
Porcentagem de Tweets relevantes em Treinamento: 12.20%
Porcentagem de Tweets Pouco relevante em Treinamento: 15.80%
Porcentagem de Tweets Não relevante em Treinamento: 69.80%


Tweets Muito relevante em Teste: 11
Tweets relevantes em Teste: 29
Tweets Pouco relevante em Teste: 59
Tweets Não relevante em Teste: 151
Tweets totais em Teste: 250


Porcentagem de Tweets Muito relevante em Teste: 4.40%
Porcentagem de Tweets relevantes em Teste: 11.60%
Porcentagem de Tweets Pouco relevante em Teste: 23.60%
Porcentagem de Tweets Não relevante em Teste: 60.40%


In [232]:
soma = treinamento(train)

In [241]:
# P(palavra)

lista = []
lista2 = []
lista_total = []
for tweet in train.Treinamento:
    lista.append(tweet)


for tweet in lista:
    a = tweet.split()
    lista2.append(a)

for tweet in lista2:
    for palavra in tweet:
        lista_total.append(palavra)

serie_test = pd.Series(lista_total)
serie_test.head()

tabela_test_relativa = serie_test.value_counts(True)



In [242]:
# Organizando o dataframe por nível de relevância
test_mr = train.loc[train['Indíce'] == 'Muito relevante',:]
test_re = train.loc[train['Indíce'] == 'Relevante',:]
test_pr = train.loc[train['Indíce'] == 'Pouco relevante',:]
test_ir = train.loc[train['Indíce'] == 'Não relevante',:]

In [245]:
# P(palvra|muito relevante)


lista = []
lista2_mr = []
palavras_mr = []
for tweet in test_mr.Treinamento:
    lista.append(tweet)

    
for tweet in lista:
    a  = tweet.split()
    lista2_mr.append(a)

for tweet in lista2_mr:
    for palavra in tweet:
        palavras_mr.append(palavra)

serie_test_mr = pd.Series(palavras_mr)
serie_test_mr.head()

tabela_test_mr = serie_test_mr.value_counts()
tabela_test_mr

tabela_test_relativa_mr = serie_test_mr.value_counts(True)
tabela_test_relativa_mr

to        0.036474
xbox      0.033435
on        0.030395
a         0.030395
and       0.027356
            ...   
issues    0.003040
tho       0.003040
lit.      0.003040
it’s      0.003040
!         0.003040
Length: 179, dtype: float64

In [246]:
prob_mr = len(palavras_mr)/len(lista_total)
prob_mr

0.027640090733428546

In [248]:
# P(palvra| relevante)
lista = []
lista2_re = []
palavras_re = []
for tweet in test_re.Treinamento:
    lista.append(tweet)

for tweet in lista:
    a  = tweet.split()
    lista2_re.append(a)

for tweet in lista2_re:
    for palavra in tweet:
        palavras_re.append(palavra)
        
serie_test_re = pd.Series(palavras_re)
serie_test_re.head()

tabela_test_re = serie_test_re.value_counts()
tabela_test_re

tabela_test_relativa_re = serie_test_re.value_counts(True)
tabela_test_relativa_re

xbox       0.027848
the        0.025316
i          0.022785
to         0.022785
a          0.018228
             ...   
stream     0.000506
1080p      0.000506
glory.     0.000506
else       0.000506
before)    0.000506
Length: 834, dtype: float64

In [249]:
prob_re = len(palavras_re)/len(lista_total)
prob_re

0.16592455683441149

In [251]:
# P(palvra|Pouco relevante)
lista = []
lista2_pr = []
palavras_pr = []
for tweet in test_pr.Treinamento:
    lista.append(tweet)

for tweet in lista:
    a  = tweet.split()
    lista2_pr.append(a)

for tweet in lista2_pr:
    for palavra in tweet:
        palavras_pr.append(palavra)
    
serie_test_pr = pd.Series(palavras_pr)
serie_test_pr.head()

tabela_test_pr = serie_test_pr.value_counts()
tabela_test_pr

tabela_test_relativa_pr = serie_test_pr.value_counts(True)
tabela_test_relativa_pr

xbox         0.030664
the          0.028961
and          0.023424
a            0.019165
i            0.019165
               ...   
glorified    0.000426
kinect       0.000426
ride,        0.000426
without      0.000426
then💀        0.000426
Length: 1010, dtype: float64

In [252]:
# P(relevante)
prob_pr = len(palavras_pr)/len(lista_total)
prob_pr

0.19726119465680922

In [254]:
#  P(palavra| irrelevante)
lista = []
lista2_ir = []
palavras_ir = []
for tweet in test_ir.Treinamento:
    lista.append(tweet)

for tweet in lista:
    a  = tweet.split()
    lista2_ir.append(a)

for tweet in lista2_ir:
    for palavra in tweet:
        palavras_ir.append(palavra)
        
serie_test_ir = pd.Series(palavras_ir)
serie_test_ir.head()

tabela_test_ir = serie_test_ir.value_counts()
tabela_test_ir

tabela_test_relativa_ir = serie_test_ir.value_counts(True)
tabela_test_relativa_ir

xbox        0.031030
the         0.024135
to          0.019032
and         0.018204
i           0.017239
              ...   
you!        0.000138
ohh         0.000138
crossing    0.000138
animal      0.000138
sale.       0.000138
Length: 2641, dtype: float64

In [255]:
#P(Irrelevante)
prob_ir = len(palavras_ir)/len(lista_total)
prob_ir

0.6091741577753508

In [256]:
# P(palavra| muito relevante) para cada palavra

dic_mr = (dict(tabela_test_relativa_mr))

lista_prob = []
for tweet in lista2_mr:
    for palavra in tweet:
        prob = dic_mr[str(palavra)]
        lista_prob.append(prob)
        
print(prob)
        

0.00303951367781155


In [257]:
# P(palavra|relevante) para cada palavra
dic_re = (dict(tabela_test_relativa_re))

lista_prob = []
for tweet in lista2_re:
    for palavra in tweet:
        prob = dic_re[str(palavra)]
        lista_prob.append(prob)

    
print(prob)

0.0005063291139240507


In [258]:
# P(palavra| Pouco relevante) para cada palavra
dic_pr = (dict(tabela_test_relativa_pr))

lista_prob = []
for tweet in lista2_pr:
    for palavra in tweet:
        prob = dic_pr[str(palavra)]
        lista_prob.append(prob)

print(prob)

0.00042589437819420784


In [259]:
# P(palavra| irrelevante) para cada palavra
dic_ir = (dict(tabela_test_relativa_ir))

lista_prob = []
for tweet in lista2_ir:
    for palavra in tweet:
        prob = dic_ir[str(palavra)]
        lista_prob.append(prob)
        
print(prob)

0.00027582402427251415


In [260]:
#Calcula Probabilidade Final para muito relevante
lista_laplace_mr = []
probf_mr = 1
dic_quant_mr = {}
quant_pal_mr = 0
pal_n_rep = []
probs_mr = []

for tweet in lista2_mr:
    for palavra in tweet:
        quant_pal_mr +=1
        
for tweet in lista2:
    for palavra in tweet:
        if palavra not in lista2_mr:
            dic_quant_mr[palavra] = 0
        elif palavra in lista2_mr:
            dic_quant_mr[palavra] += 1
    for palavra in tweet:
        if palavra not in pal_n_rep:
            pal_n_rep.append(palavra)
        prob_laplace = (dic_quant_mr[(palavra)]+1)/(len(pal_n_rep)+(quant_pal_mr))
        lista_laplace_mr.append(prob_laplace)
 
    for prob in lista_laplace_mr:
        probf_mr = probf_mr *prob
        
    probs_mr.append(probf_mr*prob_mr)    
    lista_laplace_mr =[]
    probf_mr = 1
probs_mr

[3.209211337975018e-73,
 3.8713744355602284e-92,
 2.6440916260353116e-93,
 1.8519517349611034e-70,
 3.531154940054533e-39,
 9.735905584710182e-98,
 4.314784860070742e-171,
 5.359617098772717e-143,
 1.1025283970890434e-37,
 3.163336162295658e-43,
 6.351281253211791e-27,
 1.0024138662301185e-106,
 3.125749298645512e-143,
 8.358172221299248e-58,
 1.8535664659773184e-38,
 2.268925522990908e-27,
 7.211657741876923e-89,
 7.873868341486211e-109,
 6.249376064781128e-53,
 1.6762596646073305e-78,
 9.303544496065305e-45,
 1.0757070236593587e-64,
 3.8028172540440083e-59,
 2.302928029890188e-19,
 2.0530081353150725e-96,
 2.706352387819791e-22,
 1.8747134615570595e-19,
 7.051219975213828e-97,
 1.7277596101150843e-160,
 2.151012360886781e-100,
 7.651959021299534e-14,
 3.4550271650400885e-31,
 2.0585490265248051e-112,
 4.959821370082376e-72,
 5.027628649648498e-43,
 2.908252020676739e-37,
 1.8557895220682067e-107,
 1.0769737180000824e-157,
 8.114270537330101e-76,
 1.8343314560401903e-46,
 1.8221413890

In [261]:
#Calcula Probabilidade Final para relevante
lista_laplace_re = []
probf_re = 1
dic_quant = {}
quant_pal_re = 0
probs_re = []

for tweet in lista2_re:
    for palavra in tweet:
        quant_pal_re +=1

for tweet in lista2:
    for palavra in tweet:
        if palavra not in lista2_re:
            dic_quant[palavra] = 0
        elif palavra in lista2_re:
            dic_quant[palavra] += 1
    for palavra in tweet:
        if palavra not in pal_n_rep:
            pal_n_rep.append(palavra)
        prob_laplace = (dic_quant[(palavra)]+1)/(len(pal_n_rep)+(quant_pal_re))
        lista_laplace_re.append(prob_laplace)

    for prob in lista_laplace_re:
        probf_re = probf_re *prob
        
    probs_re.append(probf_re*prob_re)    
    lista_laplace_re =[]
    probf_re = 1
    
probs_re

[2.692809669190086e-106,
 1.709146956535089e-132,
 1.709146956535089e-132,
 8.22592200229984e-99,
 6.684334304923589e-54,
 3.092359248299419e-136,
 2.7737952346725743e-237,
 4.0781059825550318e-196,
 3.694431570331267e-50,
 1.2093964727562129e-57,
 3.447509607417631e-35,
 1.012304138320534e-143,
 2.253969176558166e-192,
 2.3448840617309535e-76,
 3.694431570331267e-50,
 3.447509607417631e-35,
 1.594913978232105e-117,
 1.012304138320534e-143,
 7.163088285417643e-69,
 1.4883159041613606e-102,
 1.2093964727562129e-57,
 7.676132198668366e-84,
 2.3448840617309535e-76,
 5.820682075763278e-24,
 5.221055804940867e-125,
 1.0531358921229017e-27,
 5.820682075763278e-24,
 5.221055804940867e-125,
 2.415405853115962e-207,
 9.446455228769436e-129,
 1.778086186455741e-16,
 6.23757844656709e-39,
 1.012304138320534e-143,
 2.512832361013929e-91,
 6.684334304923589e-54,
 2.0419123289220914e-46,
 3.092359248299419e-136,
 7.378516342600021e-200,
 4.546467090671121e-95,
 1.2093964727562129e-57,
 2.18816079746

In [262]:
lista_laplace_pr = []
probf_pr = 1
dic_quant_pr = {}
quant_pal_pr = 0
probs_pr = []

for tweet in lista2_pr:
    for palavra in tweet:
        quant_pal_pr +=1

for tweet in lista2:
    for palavra in tweet:
        if palavra not in lista2_pr:
            dic_quant_pr[palavra] = 0
        elif palavra in lista2_pr:
            dic_quant_pr[palavra] += 1
    for palavra in tweet:
        if palavra not in pal_n_rep:
            pal_n_rep.append(palavra)
        prob_laplace = (dic_quant_pr[(palavra)]+1)/(len(pal_n_rep)+(quant_pal_pr))
        lista_laplace_pr.append(prob_laplace)
    
    for prob in lista_laplace_pr:
        probf_pr = probf_pr *prob
        
    probs_pr.append(probf_pr*prob_pr)    
    lista_laplace_pr =[]
    probf_pr = 1
    
probs_pr

[5.142630639524859e-107,
 2.0664326326508052e-133,
 2.0664326326508052e-133,
 1.790149725618603e-99,
 3.185029770082824e-54,
 3.5024281909335687e-137,
 5.387222656575964e-239,
 1.6245648545866894e-197,
 1.8791675643488662e-50,
 5.398355542513261e-58,
 2.2770551756705933e-35,
 1.0061557572345789e-144,
 9.584932642061468e-194,
 7.550947072388236e-77,
 1.8791675643488662e-50,
 2.2770551756705933e-35,
 2.5039710192010183e-118,
 1.0061557572345789e-144,
 2.628484675898345e-69,
 3.0341520773196664e-103,
 5.398355542513261e-58,
 2.1691890469371554e-84,
 7.550947072388236e-77,
 4.676593149240507e-24,
 7.193251994257452e-126,
 7.926429066509334e-28,
 4.676593149240507e-24,
 7.193251994257452e-126,
 7.910082601369613e-209,
 1.219195253263975e-129,
 1.6279220752506201e-16,
 3.859415551984057e-39,
 1.0061557572345789e-144,
 6.231511194878356e-92,
 3.185029770082824e-54,
 1.108708862965831e-46,
 3.5024281909335687e-137,
 2.7534997535367617e-201,
 1.0561883381149757e-95,
 5.398355542513261e-58,
 9.1

In [263]:
lista_laplace_ir = []
probf_ir = 1
dic_quant_ir = {}
quant_pal_ir = 0
probs_ir = []

#dic_quant_ir = {}
#for tweet in lista2_ir:
 #   for palavra in tweet:
 #       dic_quant_ir[palavra] = lista2_ir.count(palavra)

for tweet in lista2_ir:
    for palavra in tweet:
        quant_pal_ir +=1
for tweet in lista2:
    for palavra in tweet:
        if palavra not in lista2_ir:
            dic_quant_ir[palavra] = 0
        elif palavra in lista2_ir:
            dic_quant_ir[palavra] += 1
    for palavra in tweet:
        if palavra not in pal_n_rep:
            pal_n_rep.append(palavra)
        prob_laplace = (dic_quant_ir[str(palavra)]+1)/(len(pal_n_rep)+(quant_pal_ir))
        lista_laplace_ir.append(prob_laplace)
        
    for prob in lista_laplace_ir:
        probf_ir = probf_ir *prob
        
    probs_ir.append(probf_ir*prob_ir)    
    lista_laplace_ir =[]
    probf_ir = 1
    
probs_ir


[7.006464743529301e-114,
 4.0802644205064804e-142,
 4.0802644205064804e-142,
 8.176881296588209e-106,
 2.0659519014541818e-57,
 3.7769734522877723e-146,
 4.692948386214756e-255,
 1.0975742632068606e-210,
 2.231847839140953e-53,
 1.9123872086033344e-61,
 3.039779534160383e-37,
 3.236347743208913e-154,
 1.1857094765423714e-206,
 1.2997324133317421e-81,
 2.231847839140953e-53,
 3.039779534160383e-37,
 5.5573252180992693e-126,
 3.236347743208913e-154,
 1.5168502304899e-73,
 7.569083862434703e-110,
 1.9123872086033344e-61,
 1.1136922501040571e-89,
 1.2997324133317421e-81,
 3.832438681981811e-25,
 4.761864798647045e-134,
 3.5475688993629646e-29,
 3.832438681981811e-25,
 4.761864798647045e-134,
 8.705641653716121e-223,
 4.407909653473151e-138,
 4.472640243848989e-17,
 2.8138290605946336e-41,
 3.236347743208913e-154,
 9.542813699339993e-98,
 2.0659519014541818e-57,
 2.411065220623971e-49,
 3.7769734522877723e-146,
 1.015990246419384e-214,
 8.833484864704242e-102,
 1.9123872086033344e-61,
 1.77

In [264]:

listak = []
i = 0
while i < 500:
    if probs_mr[i]>probs_re[i] and probs_mr[i] > probs_ir[i] and probs_mr[i] > probs_pr[i]:
        listak.append('Muito relevante')
    if probs_re[i]>probs_mr[i] and probs_re[i] > probs_ir[i] and probs_re[i] > probs_pr[i]:
        listak.append('Relevante')
    if probs_pr[i]>probs_re[i] and probs_pr[i] > probs_ir[i] and probs_pr[i] > probs_mr[i]:
        listak.append('Pouco relevante')
    if probs_ir[i]>probs_re[i] and probs_ir[i] > probs_pr[i] and probs_ir[i] > probs_mr[i]:
        listak.append('Pouco relevante')
    i+=1
        
print(listak)

['Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito rele

In [265]:
train['Classificador'] = listak

In [266]:
train

,Treinamento,Indíce,Classificador
0,"this isn’t meant to be a console war tweet, it...",Não relevante,Muito relevante
1,spot the props! here's some the of models were...,Não relevante,Muito relevante
2,@jaydakid8 i had both the ps5 and xbox series ...,Não relevante,Muito relevante
3,the #playstationshowcase was a great success \...,Não relevante,Muito relevante
4,my brother finally got a new xbox but some thi...,Relevante,Muito relevante
...,...,...,...
495,@tmartn xbox or pc,Não relevante,Relevante
496,@bucksgg @nba2k xbox series s. really@hoping t...,Não relevante,Muito relevante
497,@kidsmoove xbox would rather spend money acqui...,Relevante,Muito relevante
498,@barboxbxr @majornelson you can already play i...,Não relevante,Muito relevante


In [271]:
soma = 0
for item in listak:
    if item == 'Muito relevante':
        soma+=1
        
soma

496

In [272]:
soma = 0
for item in listak:
    if item == 'Relevante':
        soma+=1
        
soma

2

In [273]:
soma = 0
for item in listak:
    if item == 'Pouco relevante':
        soma+=1
        
soma

2

In [274]:
soma = 0
for item in listak:
    if item == 'Não relevante':
        soma+=1
        
soma

0

In [281]:
print('Acerto de: {0:.2f}% em questão de tweets muito relevantes'.format(496/muito_relevantes_brutos_teste))
print('Acerto de: {0:.2f}% em questão de tweets relevantes'.format(2/relevantes_brutos_teste))
print('Acerto de: {0:.2f}% em questão de tweets pouco relevantes'.format(2/pouco_relevantes_brutos_teste))
print('Acerto de: {0:.2f}% em questão de tweets irrelevantes'.format(0/não_relevantes_brutos_teste))

Acerto de: 45.09% em questão de tweets muito relevantes
Acerto de: 0.07% em questão de tweets relevantes
Acerto de: 0.03% em questão de tweets pouco relevantes
Acerto de: 0.00% em questão de tweets irrelevantes


___
# Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [293]:
test_mr = test.loc[test['Indíce'] == 'Muito relevante',:]
test_re = test.loc[test['Indíce'] == 'Relevante',:]
test_pr = test.loc[test['Indíce'] == 'Pouco relevante',:]
test_ir = test.loc[test['Indíce'] == 'Não relevante',:]

In [294]:
lista = []
lista2 = []
lista_total = []
for tweet in test.Teste:
    lista.append(tweet)

    
for tweet in lista:
    a = tweet.split()
    lista2.append(a)

for tweet in lista2:
    for palavra in tweet:
        lista_total.append(palavra)
    
serie_test = pd.Series(lista_total)
serie_test.head()

tabela_test = serie_test.value_counts()
tabela_test

tabela_test_relativa = serie_test.value_counts(True)
tabela_test_relativa


xbox        0.036141
the         0.025214
i           0.022021
a           0.018659
to          0.017986
              ...   
pony,       0.000168
xbot        0.000168
man.        0.000168
fandoms     0.000168
launch".    0.000168
Length: 2097, dtype: float64

In [295]:
lista = []
lista2_mr = []
palavras_mr = []
for tweet in test_mr.Teste:
    lista.append(tweet)

    
for tweet in lista:
    a  = tweet.split()
    lista2_mr.append(a)

for tweet in lista2_mr:
    for palavra in tweet:
        palavras_mr.append(palavra)

serie_test_mr = pd.Series(palavras_mr)
serie_test_mr.head()

tabela_test_mr = serie_test_mr.value_counts()
tabela_test_mr

tabela_test_relativa_mr = serie_test_mr.value_counts(True)
tabela_test_relativa_mr

xbox                0.030201
to                  0.030201
the                 0.026846
you                 0.026846
for                 0.023490
                      ...   
@pocketpairgames    0.003356
not                 0.003356
perfect             0.003356
yet?                0.003356
#fixurshit          0.003356
Length: 198, dtype: float64

In [296]:
lista = []
lista2_re = []
palavras_re = []
for tweet in test_re.Teste:
    lista.append(tweet)

for tweet in lista:
    a  = tweet.split()
    lista2_re.append(a)

for tweet in lista2_re:
    for palavra in tweet:
        palavras_re.append(palavra)
        
serie_test_re = pd.Series(palavras_re)
serie_test_re.head()

tabela_test_re = serie_test_re.value_counts()
tabela_test_re

tabela_test_relativa_re = serie_test_re.value_counts(True)
tabela_test_relativa_re

the                        0.031763
xbox                       0.030668
a                          0.028478
i                          0.021906
and                        0.015334
                             ...   
any                        0.001095
park                       0.001095
lag                        0.001095
https://t.co/dvxmefx4c1    0.001095
cant                       0.001095
Length: 485, dtype: float64

In [297]:
lista = []
lista2_pr = []
palavras_pr = []
for tweet in test_pr.Teste:
    lista.append(tweet)

for tweet in lista:
    a  = tweet.split()
    lista2_pr.append(a)

for tweet in lista2_pr:
    for palavra in tweet:
        palavras_pr.append(palavra)
    
serie_test_pr = pd.Series(palavras_pr)
serie_test_pr.head()

tabela_test_pr = serie_test_pr.value_counts()
tabela_test_pr

tabela_test_relativa_pr = serie_test_pr.value_counts(True)
tabela_test_relativa_pr

xbox         0.036601
the          0.027451
i            0.022876
a            0.020915
to           0.018301
               ...   
many         0.000654
aspects      0.000654
complete,    0.000654
interface    0.000654
launch".     0.000654
Length: 734, dtype: float64

In [298]:
probs_mr

[3.209211337975018e-73,
 3.8713744355602284e-92,
 2.6440916260353116e-93,
 1.8519517349611034e-70,
 3.531154940054533e-39,
 9.735905584710182e-98,
 4.314784860070742e-171,
 5.359617098772717e-143,
 1.1025283970890434e-37,
 3.163336162295658e-43,
 6.351281253211791e-27,
 1.0024138662301185e-106,
 3.125749298645512e-143,
 8.358172221299248e-58,
 1.8535664659773184e-38,
 2.268925522990908e-27,
 7.211657741876923e-89,
 7.873868341486211e-109,
 6.249376064781128e-53,
 1.6762596646073305e-78,
 9.303544496065305e-45,
 1.0757070236593587e-64,
 3.8028172540440083e-59,
 2.302928029890188e-19,
 2.0530081353150725e-96,
 2.706352387819791e-22,
 1.8747134615570595e-19,
 7.051219975213828e-97,
 1.7277596101150843e-160,
 2.151012360886781e-100,
 7.651959021299534e-14,
 3.4550271650400885e-31,
 2.0585490265248051e-112,
 4.959821370082376e-72,
 5.027628649648498e-43,
 2.908252020676739e-37,
 1.8557895220682067e-107,
 1.0769737180000824e-157,
 8.114270537330101e-76,
 1.8343314560401903e-46,
 1.8221413890

In [299]:
probs_re

[2.692809669190086e-106,
 1.709146956535089e-132,
 1.709146956535089e-132,
 8.22592200229984e-99,
 6.684334304923589e-54,
 3.092359248299419e-136,
 2.7737952346725743e-237,
 4.0781059825550318e-196,
 3.694431570331267e-50,
 1.2093964727562129e-57,
 3.447509607417631e-35,
 1.012304138320534e-143,
 2.253969176558166e-192,
 2.3448840617309535e-76,
 3.694431570331267e-50,
 3.447509607417631e-35,
 1.594913978232105e-117,
 1.012304138320534e-143,
 7.163088285417643e-69,
 1.4883159041613606e-102,
 1.2093964727562129e-57,
 7.676132198668366e-84,
 2.3448840617309535e-76,
 5.820682075763278e-24,
 5.221055804940867e-125,
 1.0531358921229017e-27,
 5.820682075763278e-24,
 5.221055804940867e-125,
 2.415405853115962e-207,
 9.446455228769436e-129,
 1.778086186455741e-16,
 6.23757844656709e-39,
 1.012304138320534e-143,
 2.512832361013929e-91,
 6.684334304923589e-54,
 2.0419123289220914e-46,
 3.092359248299419e-136,
 7.378516342600021e-200,
 4.546467090671121e-95,
 1.2093964727562129e-57,
 2.18816079746

In [300]:
probs_pr

[5.142630639524859e-107,
 2.0664326326508052e-133,
 2.0664326326508052e-133,
 1.790149725618603e-99,
 3.185029770082824e-54,
 3.5024281909335687e-137,
 5.387222656575964e-239,
 1.6245648545866894e-197,
 1.8791675643488662e-50,
 5.398355542513261e-58,
 2.2770551756705933e-35,
 1.0061557572345789e-144,
 9.584932642061468e-194,
 7.550947072388236e-77,
 1.8791675643488662e-50,
 2.2770551756705933e-35,
 2.5039710192010183e-118,
 1.0061557572345789e-144,
 2.628484675898345e-69,
 3.0341520773196664e-103,
 5.398355542513261e-58,
 2.1691890469371554e-84,
 7.550947072388236e-77,
 4.676593149240507e-24,
 7.193251994257452e-126,
 7.926429066509334e-28,
 4.676593149240507e-24,
 7.193251994257452e-126,
 7.910082601369613e-209,
 1.219195253263975e-129,
 1.6279220752506201e-16,
 3.859415551984057e-39,
 1.0061557572345789e-144,
 6.231511194878356e-92,
 3.185029770082824e-54,
 1.108708862965831e-46,
 3.5024281909335687e-137,
 2.7534997535367617e-201,
 1.0561883381149757e-95,
 5.398355542513261e-58,
 9.1

In [301]:
probs_ir

[7.006464743529301e-114,
 4.0802644205064804e-142,
 4.0802644205064804e-142,
 8.176881296588209e-106,
 2.0659519014541818e-57,
 3.7769734522877723e-146,
 4.692948386214756e-255,
 1.0975742632068606e-210,
 2.231847839140953e-53,
 1.9123872086033344e-61,
 3.039779534160383e-37,
 3.236347743208913e-154,
 1.1857094765423714e-206,
 1.2997324133317421e-81,
 2.231847839140953e-53,
 3.039779534160383e-37,
 5.5573252180992693e-126,
 3.236347743208913e-154,
 1.5168502304899e-73,
 7.569083862434703e-110,
 1.9123872086033344e-61,
 1.1136922501040571e-89,
 1.2997324133317421e-81,
 3.832438681981811e-25,
 4.761864798647045e-134,
 3.5475688993629646e-29,
 3.832438681981811e-25,
 4.761864798647045e-134,
 8.705641653716121e-223,
 4.407909653473151e-138,
 4.472640243848989e-17,
 2.8138290605946336e-41,
 3.236347743208913e-154,
 9.542813699339993e-98,
 2.0659519014541818e-57,
 2.411065220623971e-49,
 3.7769734522877723e-146,
 1.015990246419384e-214,
 8.833484864704242e-102,
 1.9123872086033344e-61,
 1.77

In [305]:

listak = []
i = 0
while i < 250:
    if probs_mr[i]>probs_re[i] and probs_mr[i] > probs_ir[i] and probs_mr[i] > probs_pr[i]:
        listak.append('Muito relevante')
    if probs_re[i]>probs_mr[i] and probs_re[i] > probs_ir[i] and probs_re[i] > probs_pr[i]:
        listak.append('Relevante')
    if probs_pr[i]>probs_re[i] and probs_pr[i] > probs_ir[i] and probs_pr[i] > probs_mr[i]:
        listak.append('Pouco relevante')
    if probs_ir[i]>probs_re[i] and probs_ir[i] > probs_pr[i] and probs_ir[i] > probs_mr[i]:
        listak.append('Pouco relevante')
    i+=1
        
print(listak)

['Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito relevante', 'Muito rele

In [307]:
test['Classificador'] = listak

In [308]:
test

,Teste,Indíce,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Classificador
0,@swtheory66 i've got ps4 and xbox series s and...,Pouco relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
1,"@zdzisiu2007 i don't be seeing ""to xbox"" anywh...",Não relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
2,jorge henrique played ufc 4 (xbox one) in the ...,Pouco relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
3,xbox news: meet the athletes of the gaming for...,Não relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
4,controversial but if you own siege on pc and x...,Não relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
...,...,...,...,...,...,...,...,...
245,@antonlinecom @amdryzen if you guys still look...,Não relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
246,@luicrafter2 @dreddsy i cant its for xbox only,Relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
247,@psythce @larrygman90 the remake of what was o...,Não relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante
248,to celebrate the launch of the #vangaurd #beta...,Não relevante,NaN,NaN,NaN,NaN,NaN,Muito relevante


___
### Concluindo

 Ao iniciar o trabalho começamos a limpar emojis, pontuações e treinamos nosso algoritmo. 
 Após isso, calculamos a probabilidade relativa de cada palavra aparecer nos tweets, e com esses dados conseguimos calcular a probabilidade da palavra aparecer no tweet, dado a cada um dos níveis de relevância, sendo eles “Muito Relevante”, “Relevante”, “Pouco relevante”  e “relevante” Em sequência calculamos a probabilidade do tweet pertencer a uma dessas categorias, assim com esses dados aplicamos a fórmula de LaPlace para que se fosse possível adquirir a probabilidade de cada frase de cada tweet aparecer em uma das categorias mencionadas anteriormente. 
 Apesar da dedicação e diversas horas de trabalho, incluindo monitorias e atendimentos, infelizmente nosso classificador não pode ser considerado funcional, pois por algum motivo desconhecido a maioria de nossos tweets foram considerados 'Muito relevantes'. Uma de nossas principais suspeitas, foi a grande quantidade de tweets selecionados como relevantes na fase de seleção manual que podem ter causado uma grande diferença nas probabilidades e assim gerando esse resultado infortúnio.

 No nosso treinamento os resultados foram: 496 tweets Muito relevantes, 2 Relevantes, 2 Pouco relevantes e nenhum irrelevante, enquanto, no teste os 250 tweets foram classificados como Muito relevantes. Nosso classificador teve um acerto de cerca de 45% com os tweets muito relevantes, enquanto, nas outras categorias tiveram menos de 0,1% de acerto

 Na fase do teste nós mostramos que o nosso código está dando o certo e que o problema foi com a probabilidade. Os nomes das varíaveis não estão condizendo com o que signficam todas as vezes. Nesse trabalho nós conseguimos aprender a usar probabilidade e estatistica com o python.

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**